In [5]:
import sqlite3
import re

In [6]:
def extract_email_addresses(text):
    return re.findall(r'[\w\.-]+@[\w\.-]+', text)

In [7]:
def parse_domain(email):
    return email.split('@')[-1]

In [8]:
conn = sqlite3.connect('email_counts.sqlite')
cur = conn.cursor()

In [9]:
cur.execute('DROP TABLE IF EXISTS Counts')

In [11]:
file_name = input('Enter file name: ')


try:
    with open(file_name, 'r') as file:
        for line in file:
            if line.startswith('From '):
                emails = extract_email_addresses(line)
                for email in emails:
                    domain = parse_domain(email)
                    # Update counts in the database
                    cur.execute('SELECT count FROM Counts WHERE org = ? ', (domain,))
                    row = cur.fetchone()
                    if row is None:
                        cur.execute('INSERT INTO Counts (org, count) VALUES (?, 1)', (domain,))
                    else:
                        cur.execute('UPDATE Counts SET count = count + 1 WHERE org = ?', (domain,))
        conn.commit()

except FileNotFoundError:
    print("File not found:", file_name)
    quit()

# Retrieve and print the results
sqlstr = 'SELECT org, count FROM Counts ORDER BY count DESC LIMIT 10'

print("Counts:")
for row in cur.execute(sqlstr):
    print(row[0], row[1])
cur.close()

Enter file name: mbox.txt
Counts:
iupui.edu 16
umich.edu 14
uct.ac.za 12
media.berkeley.edu 8
caret.cam.ac.uk 2
gmail.com 2
